### 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab12_3 RNN with long sequence

### short sentence example

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sample = ' if you want you'
idx2char = list(set(sample)) # index -> char
char2idx = {c : i for i, c in enumerate(idx2char)} # char -> idx
print(idx2char)
print(char2idx)

['y', 'a', 'o', 'n', ' ', 'f', 't', 'w', 'u', 'i']
{'u': 8, 'f': 5, 'y': 0, 'a': 1, 'w': 7, 'o': 2, 't': 6, 'n': 3, 'i': 9, ' ': 4}


In [3]:
sample_idx = [char2idx.get(c) for c in sample] # char to index
sample_idx

[4, 9, 5, 4, 0, 2, 8, 4, 7, 1, 3, 6, 4, 0, 2, 8]

In [4]:
x_data = [sample_idx[:-1]] # X data sample (0 ~ n-1)
print(x_data) # if you want yo
y_data = [sample_idx[1:]] # Y label sample (1 ~ n)
print(y_data) #if you want you

[[4, 9, 5, 4, 0, 2, 8, 4, 7, 1, 3, 6, 4, 0, 2]]
[[9, 5, 4, 0, 2, 8, 4, 7, 1, 3, 6, 4, 0, 2, 8]]


In [5]:
## hyper-parameter setting
dic_size = len(char2idx) # RNN input size (one hot size)
hidden_size = len(char2idx) # RNN output size
num_classes = len(char2idx) # final output size (RNN or Softmax, etc.)
batch_size = 1 # one sample data , one batch
sequence_length = len(sample) - 1
learning_rate = 0.1

In [6]:
## graph setting
X = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length])
Y = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length])
X_one_hot = tf.one_hot(indices = X, depth = num_classes)
print(X_one_hot)

cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, forget_bias = 0.9, state_is_tuple = True)
print(cell.output_size, cell.zero_state)

initial_state = cell.zero_state(batch_size = batch_size, dtype = tf.float32)
outputs, states = tf.nn.dynamic_rnn(cell = cell, inputs = X_one_hot, sequence_length = [sequence_length], initial_state = initial_state)

# FC layer
outputs_for_fc = tf.reshape(tensor = outputs, shape = [-1, hidden_size])
print(outputs_for_fc)
dense_weight = tf.get_variable(name = 'dense_weight', dtype = tf.float32,
                              shape = [hidden_size, num_classes],
                              initializer = tf.contrib.layers.xavier_initializer())
bias = tf.get_variable(name = 'bias', dtype = tf.float32, shape = [num_classes], initializer = tf.random_normal_initializer())
score = tf.matmul(outputs_for_fc, dense_weight) + bias

## reshape out for sequence_loss
score_for_seq_loss = tf.reshape(tensor = score, shape = [batch_size, sequence_length, num_classes])
prediction = tf.argmax(input = score_for_seq_loss, axis = -1)
print(score_for_seq_loss)
print(prediction)
seq2seq_weight = tf.ones(shape = [batch_size, sequence_length])
seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits = score_for_seq_loss, targets = Y, weights = seq2seq_weight)
loss = tf.reduce_mean(seq2seq_loss)
opt = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Tensor("one_hot:0", shape=(?, 15, 10), dtype=float32)
10 <bound method _RNNCell.zero_state of <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell object at 0x000002DB8EBB0470>>
Tensor("Reshape:0", shape=(15, 10), dtype=float32)
Tensor("Reshape_1:0", shape=(1, 15, 10), dtype=float32)
Tensor("ArgMax:0", shape=(1, 15), dtype=int64)


In [7]:
## train
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(50):
        loss_val, _ = sess.run(fetches = [loss, opt], feed_dict = {X : x_data, Y : y_data})
        result = sess.run(prediction, feed_dict = {X : x_data})
        
        # print char using dic
        if step % 10 == 0 or step == 49:
            result_str = [idx2char[c] for c in np.squeeze(result)]
            
            print('step : {:2}, loss : {:.4f}'.format(step, loss_val), '\nprediction', ''.join(result_str))

step :  0, loss : 3.0271 
prediction wwwwwwwwwwwwwww
step : 10, loss : 0.5021 
prediction if you want you
step : 20, loss : 0.0412 
prediction if you want you
step : 30, loss : 0.0085 
prediction if you want you
step : 40, loss : 0.0037 
prediction if you want you
step : 49, loss : 0.0025 
prediction if you want you


### long sentence example

In [8]:
tf.reset_default_graph()

In [9]:
## Data pre-processing
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {c : i for i, c in enumerate(char_set)}
print(char_set)
print(char_dic)

sequence_length = 10

dataX = []
dataY = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:(i+sequence_length)]
    y_str = sentence[(i+1):(i+sequence_length+1)]
    
    if i % 20 == 0:
        print(i, x_str, ' ->', y_str)
        
    dataX.append([char_dic.get(x) for x in x_str])
    dataY.append([char_dic.get(y) for y in y_str])

print(dataX[:5])
print(dataY[:5])

['b', 'y', 'd', 'p', 'r', 'o', 'n', ' ', 'e', 'c', '.', 's', 'u', 'k', "'", ',', 'm', 'g', 'w', 'i', 'a', 'f', 't', 'h', 'l']
{'k': 13, "'": 14, 'b': 0, 'y': 1, ',': 15, 'f': 21, 'n': 6, 'd': 2, 'm': 16, 'g': 17, 'w': 18, 'i': 19, 'e': 8, 'p': 3, 'a': 20, 'r': 4, 'o': 5, ' ': 7, '.': 10, 'c': 9, 't': 22, 'h': 23, 'l': 24, 's': 11, 'u': 12}
0 if you wan  -> f you want
20  a ship, d  -> a ship, do
40 up people   -> p people t
60 o collect   ->  collect w
80 on't assig  -> n't assign
100 ks and wor  -> s and work
120 her teach   -> er teach t
140 ng for the  -> g for the 
160 mmensity o  -> mensity of
[[19, 21, 7, 1, 5, 12, 7, 18, 20, 6], [21, 7, 1, 5, 12, 7, 18, 20, 6, 22], [7, 1, 5, 12, 7, 18, 20, 6, 22, 7], [1, 5, 12, 7, 18, 20, 6, 22, 7, 22], [5, 12, 7, 18, 20, 6, 22, 7, 22, 5]]
[[21, 7, 1, 5, 12, 7, 18, 20, 6, 22], [7, 1, 5, 12, 7, 18, 20, 6, 22, 7], [1, 5, 12, 7, 18, 20, 6, 22, 7, 22], [5, 12, 7, 18, 20, 6, 22, 7, 22, 5], [12, 7, 18, 20, 6, 22, 7, 22, 5, 7]]


In [10]:
## Hyper-parameter setting
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
learning_rate = 0.1
batch_size = len(dataX)

In [11]:
## Graph setting
X = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length])
Y = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length])
X_one_hot = tf.one_hot(indices = X, depth = num_classes)
print(X_one_hot)

cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, forget_bias = 0.9, state_is_tuple = True)
print(cell.output_size, cell.zero_state)

initial_state = cell.zero_state(batch_size = batch_size, dtype = tf.float32)
outputs, states = tf.nn.dynamic_rnn(cell = cell, inputs = X_one_hot, initial_state = initial_state)

# FC layer
outputs_for_fc = tf.reshape(tensor = outputs, shape = [-1, hidden_size])
print(outputs_for_fc)
dense_weight = tf.get_variable(name = 'dense_weight', dtype = tf.float32,
                              shape = [hidden_size, num_classes],
                              initializer = tf.contrib.layers.xavier_initializer())
bias = tf.get_variable(name = 'bias', dtype = tf.float32, shape = [num_classes], initializer = tf.random_normal_initializer())
score = tf.matmul(outputs_for_fc, dense_weight) + bias

## reshape out for sequence_loss
score_for_seq_loss = tf.reshape(tensor = score, shape = [batch_size, sequence_length, num_classes])
prediction = tf.argmax(input = score_for_seq_loss, axis = -1)
print(score_for_seq_loss)
seq2seq_weight = tf.ones(shape = [batch_size, sequence_length])
seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits = score_for_seq_loss, targets = Y, weights = seq2seq_weight)
loss = tf.reduce_mean(seq2seq_loss)
opt = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)
25 <bound method _RNNCell.zero_state of <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell object at 0x000002DB94C9FF60>>
Tensor("Reshape:0", shape=(1700, 25), dtype=float32)
Tensor("Reshape_1:0", shape=(170, 10, 25), dtype=float32)


In [12]:
## train
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(3000):
    loss_val, results, _ = sess.run(fetches = [loss, prediction, opt], feed_dict = {X : dataX, Y : dataY})
    
    if step % 300 == 0 or step == 2999:
        random_number = np.random.randint(0,len(dataX))
        print('step : {:3}, loss : {:.4f}'.format(step, loss_val))
        print(''.join([char_set[x] for x in dataX[random_number]]), '->', ''.join([char_set[hat] for hat in results[random_number]]))      

step :   0, loss : 3.4583
of the sea -> wwwwwwwwww
step : 300, loss : 0.2296
r the endl ->  the endle
step : 600, loss : 0.2289
ople toget -> nle togeth
step : 900, loss : 0.2287
them tasks -> hem tasks 
step : 1200, loss : 0.2286
m tasks an ->  tasks and
step : 1500, loss : 0.2286
f you want ->  you want 
step : 1800, loss : 0.2286
't assign  -> t assign t
step : 2100, loss : 0.2286
but rather -> ui rather 
step : 2400, loss : 0.2285
ong for th ->  ' for the
step : 2700, loss : 0.2286
 them task -> toem tasks
step : 2999, loss : 0.2285
ple togeth -> ,e togethe


In [13]:
## Let's print the last char of each result to check it works
for j, result in enumerate(results):
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in result]), end='')
    else:
        print(char_set[result[-1]], end='')

f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [14]:
## Ground truth
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."